## Implementation Proposal Damiano

From here on we can write and try out code, to be later wrapped up into functions and "official" cells of this notebook.

Here is a possible scheme for implementing `square_mpo`, the function that takes as parameters an `mpo`, an upper bound `Dmax` to the bond dimension, and a number `Nsweeps` of sweeps, and returns an mpo `mpo2` representing the square of the operator represented my `mpo`. See source/square_mpo.jl for better description of what this function `square_mpo` should do. There, the easy case where (bond dimension of `mpo`)^2 <= Dmax is already implemented (no truncation is needed, so easy!). Here let's assume (bond dimension of `mpo`)^2 > Dmax, which requires truncation and actually understanding appendix D of the paper (sigh). This scheme closely follows ideas and in particular the indexing conventions of the lecture function `dmrg_2site`! Having a tab open with the implementation of `dmrg_2site` may help understand the scheme below.

- Get as parameters `mpo` to be squared, `Dmax`, and `Nsweeps`
- Initialize `mpo2` (will be output) as a copy of `mpo` (...any better ideas?) with enlarged bond dimension = `Dmax`. one has to add a bunch of zeros (1)
- by reshaping `mpo2` to an mps (by joining its physical legs), bring it to left-canonical form. Then reshape it back to the mpo form (by disjoining physical legs) (2)
- initialize `Vlr` as `Hlr` in `dmrg_2site`. roughly speaking `Vlr[l + 1]` will contain the environment tensors $V_{L/R}$ (see fig. 17c in the paper) relative to site `l` when the algorithm/sweeping is working at a couple of site that are on the right/left of site `l`.
- Fill `Vlr[2], ..., Vlr[L + 1]` with iterative contractions of `mpo`, `mpo` and `mpo2`. For that, we can use a modified version of updateLeft, say `updateLeft_mod` (3), doing what's described in fig. 17c
- Start a for cycle with the index `itS = (1:Nsweeps)`
- RIGHT TO LEFT SWEEP: `for itL = (L:-1:2)`, update (4) as in appendix D `mpo2[itL-1]` and `mpo2[itL]`. This involves `Vlr[itL-1]` as left environment $V_L$ and `Vlr[itL + 2]` as right environment $V_R$. It also involves an SVD (5) to get `mpo2[itL-1]` and `mpo2[itL]` out of the 2-site optimized tensor ($C_i C_{i+1}$ in the paper). In particular, the updated `mpo2[itL]` must be an "MPO-right isometry", so that the updated `mpo2`  will be "MPO-site canonical" with new isometry center at site `itL - 1` (still need to really get the details of all that...). After that, update `Vlr[itL + 1]` using `updateLeft_mod` (and an index permutation since we are "updating leftwards")
- LEFT TO RIGHT SWEEP: similar to above: `for itL = (1:L-1)`, update `mpo2[itL]` and `mpo2[itL+1]` using `Vlr[itL] ` and `Vlr[itL + 3]`. Then update `Vlr[itL + 1]` using `updateLeft_mod`
- end of cycle with index itS
- `mpo2` should now approximately represent the square of the operator represented by `mpo`!! ...for big enough `Dmax` and `Nsweeps`

In [ ]:
Vlr = Array{Any}(undef,1,L+2); # L is the chain length
Vlr[1] = reshape([1],(1,1,1)); # watch out, the dummy site on the left of the first site makes indices shift by one!
Vlr[end] = reshape([1],(1,1,1));

Some "isolated" functions that we can isolate and test before implementing the whole scheme are the following:

(1) write & test a function to enlarge bond dim. of an MPO by "adding zeros"

(2) understand how to bring an mpo to left-canonical form (what does this really mean? whould we really just transform the mpo into an mps by joining physical legs, getting canonical form of mps, and get back to mpo form?)

(3) write & test `updateLeft_mod` following fig 17c. use/copy `updateLeft`

(4) write & test a function for the two-site optimization (without svd), see appendix D

(5) write & test a function to SVD a 2-site, 6-leg MPO tensor. use/copy `svd`, `svdleft` and `svdright` from lecture 

## Implementation Proposal Matthias

